In [2]:
import sys, importlib as impL
sys.path.insert(1,'/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/vae_torch')
sys.path.insert(1,'/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery')
import vae_torch_model as vtm
import vae_torch as vt
#from data_classes import khs_dataset_v2
import vae_scripts as vs
import sae_torch as st
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import vae_utils as vu
import pandas as pd
import helperFuncs as funcH

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option("display.precision", 3)
pd.set_option('display.max_rows', None)

In [3]:
def get_acc_err(experiments_folder, exp_base_name,cf_int,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=1):
    ae_fold_name = os.path.join(experiments_folder, exp_base_name + str(cf_int).zfill(z_fill_int))
    ae_f_name = os.path.join(ae_fold_name, ae_f_name_base)
    vfz = np.load(ae_f_name, allow_pickle=True)
    loss_log_dict = {}
    n = 0
    loss_log_dict[data_log_key] = vfz.item().get(data_log_key)
    if loss_log_dict[data_log_key] is None:
        print("cf("+str(cf_int)+") --> loss_log_dict"+str(data_log_key)+"] is none")
        return None, None, None
    n = len(loss_log_dict[data_log_key])
    if verbose>0:
        print(str(cf_int), ', ', data_log_key, " - log is loaded with len: ", n)
    if loss_key not in loss_log_dict[data_log_key][0].keys():
        return n, None, None

    if loss_key in loss_log_dict[data_log_key][0]:
        vec_len = len(loss_log_dict[data_log_key])
        los_vec_cur = [loss_log_dict[data_log_key][l][loss_key] for l in range(0, vec_len)]
        label_str = str(cf_int) + '_' + data_log_key + '_' + loss_key
        if verbose>0:
            print(label_str, los_vec_cur[-3:], "\nmax({:4.2f}),min({:4.2f})".format(np.max(los_vec_cur), np.min(los_vec_cur)))
        return n, np.min(los_vec_cur), np.max(los_vec_cur)
    return None, None, None

In [4]:
impL.reload(vu)
experiments_folder='/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/experiments/FM/'
config_folder = '/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/configs/'
z_fill_int=2
loss_key='reconstruction' # sparsity, bottleneck_kmeans, bottleneck_act
data_log_keys=['tr_te']  # ['tr_va', 'va', 'te']
exp_base_name='exp_conv_ae_simple_is28_cf'
ae_f_name_base='ae_ft_conv_ae_simple_is28.npy'
#cf_int_arr=[201,202,203,204,205,206,207,208,209,210,211,212,213,214,216,217,218]
#cf_int_arr=[201,204,205,207,213,214]
#cf_int_arr=[201,204,205,216,217,218]
#cf_int_arr=[220,221,222,223,224,225,226,227]
cf_int_arr=np.asarray(range(220,249), dtype=int)
df1 = pd.DataFrame(index=cf_int_arr, columns=['spMethod','spW','spR','kl_div','bact','sigm','logsm','lact','rcErr','lr','rcRed'])
df2 = pd.DataFrame(index=cf_int_arr, columns=['n','bActMin','bActMax','bErrMin','bErrMax','kmMin','kmMax','rErrMax','rErrMin'])
df3 = pd.DataFrame(index=cf_int_arr, columns=['n','bActMin','bActMax','bErrMin','bErrMax','kmMin','kmMax','rErrMax','rErrMin'])
df4 = pd.DataFrame(index=cf_int_arr, columns=['kl_div','bact','sigm','logsm','lr', 'n','bActMax','bErrMin','bErrDif','kmMax','rErrDif'])
for i in range(0,len(cf_int_arr)):
    cf = cf_int_arr[i]
    #load model yaml as config
    config_file = os.path.join(config_folder, 'conf_autoencoder_'+str(cf)+'.yaml')
    CONF_PARAMS_ = funcH.CustomConfigParser(config_file=config_file)
    df1.iloc[i]['spW'] = float(CONF_PARAMS_.MODEL.SPARSITY_WEIGHT)
    if df1.iloc[i]['spW'] > 0:
        df1.iloc[i]['spMethod'] = CONF_PARAMS_.MODEL.SPARSITY_ERROR
        df1.iloc[i]['spR'] = CONF_PARAMS_.MODEL.SPARSITY_REDUCTION
    else:
        df1.iloc[i]['spW'] = None
    df1.iloc[i]['kl_div'] = CONF_PARAMS_.MODEL.KL_DIV_RHO if 'kl_' in str(CONF_PARAMS_.MODEL.SPARSITY_ERROR) else None
    df4.iloc[i]['kl_div'] = df1.iloc[i]['kl_div']
    try:
        df1.iloc[i]['bact'] = (CONF_PARAMS_.MODEL.LAYERS.encoder.l04_act).replace('type: ','').replace(',dim:','')
        df4.iloc[i]['bact'] = df1.iloc[i]['bact']
    except:
        pass
    df1.iloc[i]['sigm'] = CONF_PARAMS_.MODEL.KL_SIGMOID
    df4.iloc[i]['sigm'] = df1.iloc[i]['sigm']
    df1.iloc[i]['logsm'] = CONF_PARAMS_.MODEL.KL_LOGSOFTMAX
    df4.iloc[i]['logsm'] = df1.iloc[i]['logsm']
    try:
        df1.iloc[i]['lact'] = CONF_PARAMS_.MODEL.LAYERS.decoder.l01_act.replace('type: ','')
    except:
        pass
    df1.iloc[i]['rcErr'] = CONF_PARAMS_.MODEL.RECONSTRUCTION_ERROR_FUNCTION
    df1.iloc[i]['lr'] = CONF_PARAMS_.MODEL.LEARNING_RATE
    df4.iloc[i]['lr'] = df1.iloc[i]['lr']
    df1.iloc[i]['rcRed'] = CONF_PARAMS_.MODEL.RECONSTRUCTION_ERROR_REDUCTION
    
    try:
        data_log_key = 'tr_te'
        loss_key = 'reconstruction' # reconstruction bottleneck_kmeans bottleneck_act sparsity
        n, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df2.iloc[i]['n'] = n
        df4.iloc[i]['n'] = df2.iloc[i]['n']

        df2.iloc[i]['rErrMin'] = mn
        df2.iloc[i]['rErrMax'] = mx
        df4.iloc[i]['rErrDif'] = df2.iloc[i]['rErrMax']-df2.iloc[i]['rErrMin']
            
        loss_key = 'bottleneck_kmeans' # bottleneck_act sparsity
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df2.iloc[i]['kmMin'] = mn
        df2.iloc[i]['kmMax'] = mx

        loss_key = 'bottleneck_act' #  sparsity
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df2.iloc[i]['bActMin'] = mn
        df2.iloc[i]['bActMax'] = mx if mx!=mn else '_n*c_'
        df4.iloc[i]['bActMax'] = df2.iloc[i]['bActMax']

        loss_key = 'sparsity' #  
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df2.iloc[i]['bErrMin'] = mn
        df2.iloc[i]['bErrMax'] = mx if mx!=mn else '_n*c_'
        df4.iloc[i]['bErrMin'] = df2.iloc[i]['bErrMin']

        try:
            df4.iloc[i]['bErrDif'] = df2.iloc[i]['bErrMax']-df2.iloc[i]['bErrMin']
        except:
            df4.iloc[i]['bErrDif'] = 0
        data_log_key = 'te'
        loss_key = 'reconstruction' # reconstruction bottleneck_kmeans bottleneck_act sparsity
        n, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df3.iloc[i]['n'] = n

        df3.iloc[i]['rErrMin'] = mn
        df3.iloc[i]['rErrMax'] = mx

        loss_key = 'bottleneck_kmeans' # bottleneck_act sparsity
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df3.iloc[i]['kmMin'] = mn
        df3.iloc[i]['kmMax'] = mx
        df4.iloc[i]['kmMax'] = df2.iloc[i]['kmMax']

        loss_key = 'bottleneck_act' #  sparsity
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df3.iloc[i]['bActMin'] = mn
        df3.iloc[i]['bActMax'] = mx if mx!=mn else '_n*c_'

        loss_key = 'sparsity' #  
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df3.iloc[i]['bErrMin'] = mn
        df3.iloc[i]['bErrMax'] = mx if mx!=mn else '_n*c_'
    except:
        print("no data available for d3 - ", cf)

In [5]:
pd.set_option("display.precision", 2)
print('experiments explanations')
print(df1)

experiments explanations
          spMethod    spW        spR  kl_div  bact   sigm  logsm lact rcErr      lr rcRed
220  kl_divergence  0.001  batchmean  0.0001  ReLu   True   True  NaN   MSE    0.01  mean
221  kl_divergence  0.001  batchmean  0.0001  ReLu   True  False  NaN   MSE    0.01  mean
222  kl_divergence  0.001  batchmean  0.0001  ReLu  False  False  NaN   MSE    0.01  mean
223  kl_divergence  0.001  batchmean  0.0001  ReLu  False   True  NaN   MSE    0.01  mean
224  kl_divergence  0.001  batchmean  0.0001   NaN   True   True  NaN   MSE    0.01  mean
225  kl_divergence  0.001  batchmean  0.0001   NaN   True  False  NaN   MSE    0.01  mean
226  kl_divergence  0.001  batchmean  0.0001   NaN  False  False  NaN   MSE    0.01  mean
227  kl_divergence  0.001  batchmean  0.0001   NaN  False   True  NaN   MSE    0.01  mean
228  kl_divergence  0.001  batchmean  0.0001  ReLu   True   True  NaN   MSE   0.001  mean
229  kl_divergence  0.001  batchmean  0.0001  ReLu   True  False  NaN   MSE

In [6]:
print('tr_te dataset results')
pd.set_option("display.precision", 5)
print(df2)

tr_te dataset results
       n bActMin bActMax     bErrMin  bErrMax   kmMin   kmMax rErrMax rErrMin
220   10      10   _n*c_     0.20336  0.30914      10  19.812  50.796  47.029
221   10      10   _n*c_     -1874.5  -1871.6    8.56  13.008  50.082  47.497
222   10      10   _n*c_     -1705.2  -1696.6      10   13.02  53.554  48.151
223   10      10   _n*c_      3.1483   5.3453   8.075   34.23  50.265  47.605
224   10      10   _n*c_     0.33957  0.39402      10  16.293  50.158  47.569
225   10      10   _n*c_     -1872.2  -1869.4      10  17.693  51.232  47.326
226   10      10   _n*c_     -1697.2  -1687.7  5.8517  15.805  51.149  47.193
227   10      10   _n*c_      5.4157   6.1237      10  20.632  51.185  47.148
228   10      10   _n*c_      1.6792   6.3684  15.497  21.088  39.204   38.27
229   10      10   _n*c_       -2008  -1960.1  14.718  20.472  39.469  38.468
230   10      10   _n*c_ -8.8827e+12  -2864.2  16.375  20.825  96.826  38.334
231   10      10  16.915 -2.5379e-05   55.

In [7]:
print('te dataset results')
print(df3)

te dataset results
       n bActMin bActMax     bErrMin   bErrMax  kmMin  kmMax rErrMax rErrMin
220   10      10   _n*c_    0.034255  0.052072     10  23.89  8.5156  7.8816
221   10      10   _n*c_     -315.75   -315.25     10  12.88  8.3941  7.9609
222   10      10   _n*c_     -287.22   -285.79     10  14.61  8.9803  8.0696
223   10      10   _n*c_      0.5303   0.90038     10   29.8  8.4276  7.9799
224   10      10   _n*c_    0.057198  0.066369     10  27.17  8.4094  7.9729
225   10      10   _n*c_     -315.36   -314.89     10  20.18   8.586  7.9339
226   10      10   _n*c_     -285.88   -284.28     10   15.8  8.5754  7.9101
227   10      10   _n*c_     0.91223    1.0315     10  25.09  8.5766  7.9016
228   10      10   _n*c_     0.28382    1.0741  15.77  22.33  6.5801  6.4221
229   10      10   _n*c_     -338.26   -330.19  14.79  19.74  6.6235  6.4551
230   10      10   _n*c_ -1.4967e+12   -482.42   15.5   21.2  16.269  6.4339
231   10      10    21.8 -4.4564e-06    9.2733   15.8  20

In [8]:
print('picky results')
print(df4)

picky results
     kl_div  bact   sigm  logsm      lr    n bActMax     bErrMin     bErrDif   kmMax  rErrDif
220  0.0001  ReLu   True   True    0.01   10   _n*c_     0.20336     0.10578  19.812    3.767
221  0.0001  ReLu   True  False    0.01   10   _n*c_     -1874.5      2.9479  13.008   2.5853
222  0.0001  ReLu  False  False    0.01   10   _n*c_     -1705.2      8.5145   13.02   5.4032
223  0.0001  ReLu  False   True    0.01   10   _n*c_      3.1483       2.197   34.23   2.6604
224  0.0001   NaN   True   True    0.01   10   _n*c_     0.33957    0.054449  16.293   2.5886
225  0.0001   NaN   True  False    0.01   10   _n*c_     -1872.2      2.7629  17.693   3.9059
226  0.0001   NaN  False  False    0.01   10   _n*c_     -1697.2      9.4812  15.805   3.9562
227  0.0001   NaN  False   True    0.01   10   _n*c_      5.4157     0.70804  20.632    4.037
228  0.0001  ReLu   True   True   0.001   10   _n*c_      1.6792      4.6892  21.088  0.93429
229  0.0001  ReLu   True  False   0.001   10  